In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import CategoricalCrossentropy

from sklearn.metrics import classification_report,confusion_matrix
#import seaborn as sn
from tensorflow.keras.preprocessing import image

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('seg_train/',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('seg_test/',
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            shuffle=False,
                                            class_mode='categorical')

FileNotFoundError: ignored

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
labels = {value: key for key, value in training_set.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

NameError: ignored

In [ ]:
def create_model():
    model = Sequential([
        Conv2D(filters=128, kernel_size=(5, 5), padding='valid', input_shape=(256, 256, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=32, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        Flatten(),

        Dense(units=256, activation='relu'),
        Dropout(0.5),
        Dense(units=6, activation='softmax')
    ])

    return model

In [ ]:
cnn_model = create_model()

In [ ]:
cnn_model.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
history = cnn_model.fit(training_set, epochs=25, validation_data=test_set,
                       verbose=1)

Epoch 1/25
439/439 [==============================] - 1093s 2s/step - loss: 3.8844 - accuracy: 0.4936 - val_loss: 1.4097 - val_accuracy: 0.5447
Epoch 2/25
439/439 [==============================] - 1089s 2s/step - loss: 1.2234 - accuracy: 0.5959 - val_loss: 0.9577 - val_accuracy: 0.6823
Epoch 3/25
439/439 [==============================] - 1091s 2s/step - loss: 0.9831 - accuracy: 0.6523 - val_loss: 0.9417 - val_accuracy: 0.6867
Epoch 4/25
439/439 [==============================] - 1090s 2s/step - loss: 0.8737 - accuracy: 0.6873 - val_loss: 0.9439 - val_accuracy: 0.7030
Epoch 5/25
439/439 [==============================] - 1089s 2s/step - loss: 0.8331 - accuracy: 0.7031 - val_loss: 0.8382 - val_accuracy: 0.7207
Epoch 6/25
439/439 [==============================] - 1093s 2s/step - loss: 0.7755 - accuracy: 0.7260 - val_loss: 1.7542 - val_accuracy: 0.5030
Epoch 7/25
439/439 [==============================] - 1091s 2s/step - loss: 0.7396 - accuracy: 0.7349 - val_loss: 1.0148 - val_accuracy:

In [ ]:
test_loss, test_accuracy = cnn_model.evaluate(test_set, batch_size=256)

94/94 [==============================] - 51s 537ms/step - loss: 0.5306 - accuracy: 0.8190


In [ ]:
predictions = cnn_model.predict(test_set)

In [ ]:
print(f"Test Loss:     {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Test Loss:     0.53060382604599
Test Accuracy: 0.8190000057220459


In [ ]:
y_pred = np.argmax(predictions, axis=1)
y_true = test_set.classes

In [ ]:
print(classification_report(y_true, y_pred, target_names=labels.values()))

              precision    recall  f1-score   support

   buildings       0.86      0.78      0.82       437
      forest       0.96      0.97      0.96       474
     glacier       0.86      0.63      0.73       553
    mountain       0.66      0.86      0.75       525
         sea       0.84      0.80      0.82       510
      street       0.82      0.90      0.85       501

    accuracy                           0.82      3000
   macro avg       0.83      0.82      0.82      3000
weighted avg       0.83      0.82      0.82      3000

